In [1]:
from datetime import datetime, timedelta
from json import loads, JSONDecodeError
from logging import (
    basicConfig,
    CRITICAL,
    ERROR,
    FileHandler,
    getLogger,
    INFO,
    log,
    StreamHandler,
)
from os import getenv, makedirs, path
from re import findall
from time import localtime, sleep, strftime, time
from traceback import TracebackException

from dotenv import load_dotenv
from openpyxl import load_workbook, Workbook
from pandas import DataFrame
from seleniumwire import webdriver
from seleniumwire.utils import decode
from selenium.common.exceptions import (
    NoSuchElementException,
    StaleElementReferenceException,
    ElementNotInteractableException,
)
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.remote_connection import LOGGER as seleniumLogger
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from urllib3.connectionpool import log as urllibLogger
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
class Errores:
    """
    Representa a los errores ocurridos durante la ejecución de un scraper

    ...

    Attributes
    ----------
    errores : dict
        Conjunto de datos que contiene toda información de los errores ocurridos durante la ejecución del scraper

    Methods
    -------
    agregar_error(error, enlace):
        Agrega la información de un error al diccionario de datos errores
    """

    def __init__(self):
        """
        Genera todos los atributos para el objeto Errores
        """
        self._errores = {
            "Clase": [],
            "Mensaje": [],
            "Linea de Error": [],
            "Codigo Error": [],
            "Publicacion": [],
        }

    @property
    def errores(self):
        """Retorna el valor actual del diccionario de datos errores"""
        return self._errores

    def agregar_error(self, error, enlace):
        """
        Agrega la información de un error al diccionario de datos errores

        Parameters
        ----------
        error: Exception
            Objeto de tipo excepción ocurrida durante la ejecución del scraper
        enlace: str
            Enlace de la publicación de la página facebook marketplace

        Returns
        -------
        None
        """
        log(ERROR, error)
        traceback_error = TracebackException.from_exception(error)
        error_stack = traceback_error.stack[0]
        self._errores["Clase"].append(traceback_error.exc_type)
        self._errores["Mensaje"].append(traceback_error._str)
        self._errores["Linea de Error"].append(error_stack.lineno)
        self._errores["Codigo Error"].append(error_stack.line)
        self._errores["Publicacion"].append(enlace)

In [3]:
class Dataset:
    """
    Representa al conjunto de datos generado por el scraper

    ...

    Attributes
    ----------
    dataset : dict
        Conjunto de datos que contiene toda información extraída de una categoría de la página de facebook marketplace

    Methods
    -------
    agregar_data():
        Agrega la información de una publicación al diccionario de datos dataset
    """

    def __init__(self):
        """
        Genera todos los atributos para el objeto Dataset
        """
        self._dataset = {
            "Fecha Extraccion": [],
            "titulo_marketplace": [],
            "tiempo_creacion": [],
            "tipo_delivery": [],
            "descripcion": [],
            "disponible": [],
            "vendido": [],
            "fecha_union_vendedor": [],
            "cantidad": [],
            "precio": [],
            "tipo_moneda": [],
            "amount_with_concurrency": [],
            "latitud": [],
            "longitud": [],
            "locacion": [],
            "locacion_id": [],
            "name_vendedor": [],
            "tipo_vendedor": [],
            "id_vendedor": [],
            "enlace": [],
        }

    @property
    def dataset(self):
        """Retorna el valor actual del diccionario de datos dataset"""
        return self._dataset

    def agregar_data(self, item, fecha_extraccion, enlace):
        """
        Agrega la información de una publicación al dataset

        Parameters
        ----------
        item: dict
            Conjunto de datos que contiene toda la información de una publicación
        fecha_extraccion: str
            Fecha actual en la que se creó una publicación en formato %d/%m/%Y
        enlace: str
            Enlace de la publicación de la página facebook marketplace

        Returns
        -------
        None
        """
        self._dataset["titulo_marketplace"].append(
            item.get("marketplace_listing_title")
        )
        self._dataset["tiempo_creacion"].append(item.get("creation_time"))
        self._dataset["disponible"].append(item.get("is_live"))
        self._dataset["vendido"].append(item.get("is_sold"))
        self._dataset["cantidad"].append(item.get("listing_inventory_type"))
        self._dataset["name_vendedor"].append(
            item.get("story").get("actors")[0].get("name")
        )
        self._dataset["tipo_vendedor"].append(
            item.get("story").get("actors")[0]["__typename"]
        )
        self._dataset["id_vendedor"].append(item.get("story").get("actors")[0]["id"])
        self._dataset["locacion_id"].append(item.get("location_vanity_or_id"))
        self._dataset["latitud"].append(item.get("location", {}).get("latitude"))
        self._dataset["longitud"].append(item.get("location", {}).get("longitude"))
        self._dataset["precio"].append(item.get("listing_price", {}).get("amount"))
        self._dataset["tipo_moneda"].append(
            item.get("listing_price", {}).get("currency")
        )
        self._dataset["amount_with_concurrency"].append(
            item.get("listing_price", {}).get("amount_with_offset_in_currency")
        )
        self._dataset["tipo_delivery"].append(item.get("delivery_types", [None])[0])
        self._dataset["descripcion"].append(
            item.get("redacted_description", {}).get("text")
        )
        self._dataset["fecha_union_vendedor"].append(
            item.get("marketplace_listing_seller", {}).get("join_time")
        )
        data = item.get("location_text", {})
        if data:
            data = data.get("text")
        self._dataset["locacion"].append(data)
        self._dataset["Fecha Extraccion"].append(fecha_extraccion)
        self._dataset["enlace"].append(enlace)

In [4]:
class Tiempo:
    """
    Representa el tiempo que se demora el scraper en extraer la información

    ...

    Attributes
    ----------
    start : float
        Hora actual en segundos
    hora_inicio : str
        Hora de inicio de la ejecución del scraper en formato %H:%M:%S
    fecha : str
        Fecha de las publicaciones a extraer en formato %d/%m/%Y
    hora_fin : str
        Hora de término de la ejecución del scraper en formato %H:%M:%S
    cantidad : int
        Cantidad de publicaciones extraídas de la página de facebook marketplace
    cantidad_real: int
        Cantidad real de publicaciones extraídas de la página de facebook marketplace
    tiempo : str
        Tiempo de ejecución del scraper en formato %d days, %H:%M:%S
    productos_por_min : float
        Cantidad de publicaciones que puede extraer el scraper en un minuto
    productos_por_min_real : float
        Cantidad real de publicaciones que puede extraer el scraper en un minuto
    num_error : int
        Cantidad de errores ocurridos durante la ejecución del scraper

    Methods
    -------
    set_param_final():
        Establece los parámetros finales cuando se termina de ejecutar el scraper
    """

    def __init__(self, fecha_actual):
        """
        Genera todos los atributos para el objeto Tiempo

        Parameters
        ----------
        fecha_actual: str
            Fecha en la que se ejecuta el scraper
        """
        self._start = time()
        self._hora_inicio = strftime("%H:%M:%S", localtime(self._start))
        log(INFO, f"Hora de inicio: {self._hora_inicio}")
        self._fecha = fecha_actual.strftime("%d/%m/%Y")
        self._hora_fin = None
        self._cantidad = None
        self._cantidad_real = None
        self._tiempo = None
        self._productos_por_min = None
        self._productos_por_min_real = None
        self._num_error = None

    @property
    def cantidad(self):
        """Retorna el valor actual o asigna un nuevo valor del atributo cantidad"""
        return self._cantidad

    @property
    def cantidad_real(self):
        """Retorna el valor actual o asigna un nuevo valor del atributo cantidad_real"""
        return self._cantidad_real

    @property
    def fecha(self):
        """Retorna el valor actual del atributo fecha"""
        return self._fecha

    @property
    def num_error(self):
        """Retorna el valor actual o asigna un nuevo valor del atributo num_error"""
        return self._num_error

    @cantidad.setter
    def cantidad(self, cantidad):
        self._cantidad = cantidad

    @cantidad_real.setter
    def cantidad_real(self, cantidad_real):
        self._cantidad_real = cantidad_real

    @num_error.setter
    def num_error(self, num_error):
        self._num_error = num_error

    def set_param_final(self):
        """
        Establece parametros finales para medir el tiempo de ejecución del scraper

        Parameters
        ----------
        None

        Returns
        -------
        None
        """
        end = time()
        self._hora_fin = strftime("%H:%M:%S", localtime(end))
        log(INFO, f"Productos Extraídos: {self._cantidad}")
        log(INFO, f"Hora Fin: {self._hora_fin}")
        total = end - self._start
        self._tiempo = str(timedelta(seconds=total)).split(".")[0]
        self._productos_por_min = round(self._cantidad / (total / 60), 2)
        self._productos_por_min_real = round(self._cantidad_real / (total / 60), 2)

In [5]:
class ScraperFb:
    """
    Representa a un bot para hacer web scraping en fb marketplace

    ...

    Attributes
    ----------
    tiempo : Tiempo
        Objeto de la clase Tiempo que maneja información del tiempo de ejecución del scraper
    driver: webdriver.Chrome
        Objeto de la clase webdriver que maneja un navegador para hacer web scraping
    wait : WebDriverWait
        Objeto de la clase WebDriverWait que maneja el Tiempo de espera durante la ejecución del scraper
    errores : Errores
        Objeto de la clase Errores que maneja información de los errores ocurridos durante la ejecución del scraper
    data : Dataset
        Objeto de la clase Dataset que maneja información de las publicaciones extraídas por el scraper

    Methods
    -------
    iniciar_sesion():
        Iniciar sesión en facebook usando un usuario y contraseña
    mapear_datos(url):
        Mapea y extrae los datos de las publicaciones de una categoría
    guardar_datos(dataset, filetype, folder, filename):
        Guarda los datos o errores obtenidos durante la ejecución del scraper
    guardar_tiempos(filename, sheet_name):
        Guarda la información del tiempo de ejecución del scraper
    """

    def __init__(self, fecha_actual):
        """
        Genera todos los atributos para el objeto ScraperFb

        Parameters
        ----------
        fecha_actual: str
            Fecha en la que se ejecuta el scraper
        """
        log(INFO, "Inicializando scraper")
        self._tiempo = Tiempo(fecha_actual)
        chrome_options = webdriver.ChromeOptions()
        prefs = {"profile.default_content_setting_values.notifications": 2}
        chrome_options.add_experimental_option("prefs", prefs)
        self._driver = webdriver.Chrome(
            chrome_options=chrome_options,
            service=Service(ChromeDriverManager().install()),
        )
        self._wait = WebDriverWait(self._driver, 10)
        self._errores = Errores()
        self._data = Dataset()

    @property
    def data(self):
        """Retorna el valor actual del atributo data"""
        return self._data

    @property
    def errores(self):
        """Retorna el valor actual del atributo errores"""
        return self._errores

    def iniciar_sesion(self):
        """
        Inicia sesión en una página web usando un usuario y contraseña

        Parameters
        ----------
        None

        Returns
        -------
        None
        """
        log(INFO, "Iniciando sesión")
        self._driver.get("https://www.facebook.com/")
        self._driver.maximize_window()
        username = self._wait.until(EC.presence_of_element_located((By.ID, "email")))
        password = self._wait.until(EC.presence_of_element_located((By.ID, "pass")))
        username.clear()
        password.clear()
        username.send_keys(getenv("FB_USERNAME"))
        password.send_keys(getenv("FB_PASSWORD"))
        self._wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "button[name='login']"))
        ).click()
        log(INFO, "Inicio de sesión con éxito")

    def mapear_datos(self, url):
        """
        Mapea y extrae los datos de las publicaciones de una categoría

        Parameters
        ----------
        url: str
            Link de la página de una categoría en facebook marketplace

        Returns
        -------
        None
        """
        sleep(10)
        log(INFO, "Accediendo a la URL")
        self._driver.execute_script("window.open('about:blank', 'newtab');")
        self._driver.switch_to.window("newtab")
        self._driver.get(url)

        sleep(8)
        log(INFO, "Mapeando Publicaciones")
        ropa = self._driver.find_elements(
            By.XPATH, '//*[@class="xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3"]'
        )
        fecha_publicacion = fecha_extraccion = int(
            datetime.strptime(self._tiempo.fecha, "%d/%m/%Y").timestamp()
        )
        fecha_flag = fecha_extraccion + 86400
        i = 0
        e = 0
        del self._driver.requests

        while fecha_publicacion >= fecha_extraccion:
            try:
                log(INFO, f"Scrapeando item {i + 1}")
                try:
                    enlace = findall(
                        "(.*)\/\?",
                        ropa[i]
                        .find_element(By.XPATH, ".//ancestor::a")
                        .get_attribute("href"),
                    )[0]
                except NoSuchElementException as error:
                    enlace = None
                    self._errores.agregar_error(error, enlace)
                ropa[i].click()
                sleep(5)
                for request in self._driver.requests:
                    if not request.response or "graphql" not in request.url:
                        continue

                    body = decode(
                        request.response.body,
                        request.response.headers.get("Content-Encoding", "identity"),
                    )
                    decoded_body = body.decode("utf-8")
                    json_data = loads(decoded_body)

                    if "prefetch_uris_v2" not in json_data["extensions"]:
                        continue

                    fecha_publicacion = json_data["data"]["viewer"][
                        "marketplace_product_details_page"
                    ]["target"]["creation_time"]
                    if fecha_publicacion < fecha_flag:
                        dato = json_data["data"]["viewer"][
                            "marketplace_product_details_page"
                        ]["target"]
                        log(INFO, f"{dato['marketplace_listing_title']}")
                        self._data.agregar_data(dato, self._tiempo.fecha, enlace)
                        log(INFO, f"Item {i + 1} scrapeado con éxito")
                    break
                self._driver.execute_script("window.history.go(-1)")

            except (
                NoSuchElementException,
                ElementNotInteractableException,
                StaleElementReferenceException,
            ) as error:
                self._errores.agregar_error(error, enlace)
                e += 1

            except (KeyError, JSONDecodeError) as error:
                self._errores.agregar_error(error, enlace)
                e += 1
                self._driver.execute_script("window.history.go(-1)")

            except Exception as error:
                self._errores.agregar_error(error, enlace)
                e += 1
                i += 1
                log(CRITICAL, "Se detuvo inesperadamente el programa")
                log(CRITICAL, f"Causa:\n{error}")
                break
            finally:
                i += 1
                if i == len(ropa):
                    self._driver.execute_script(
                        "window.scrollTo(0, document.body.scrollHeight)"
                    )
                    sleep(7)
                    ropa = self._driver.find_elements(
                        By.XPATH,
                        '//*[@class="xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3"]',
                    )
                sleep(2)
                del self._driver.requests
                log(
                    INFO,
                    "-------------------------------------------------------------------",
                )
        self._tiempo.cantidad_real = i - e
        self._tiempo.num_error = e
        log(INFO, f"Se halló {e} errores")
        log(INFO, "Fin de la extraccion")

    def guardar_datos(
        self,
        dataset,
        filetype="Data",
        folder="Data//datos_obtenidos",
        filename="fb_data",
    ):
        """
        Guarda los datos o errores obtenidos durante la ejecución del scraper

        Parameters
        ----------
        dataset: dict
            Conjunto de datos extraídos por el scraper
        filetype: str
            Indica si la información proviene de los datos o de los errores
        folder: str
            Ruta del archivo
        filename: str
            Nombre del archivo

        Returns
        -------
        None
        """
        log(INFO, f"Guardando {filetype}")
        df_fb_mkp_ropa = DataFrame(dataset)

        if len(df_fb_mkp_ropa) == 0:
            log(
                INFO,
                f"El archivo de tipo {filetype} no se va a guardar por no tener información",
            )
            return

        if filetype == "Data":
            df_fb_mkp_ropa.drop(len(df_fb_mkp_ropa) - 1, axis=0, inplace=True)
            cantidad = len(df_fb_mkp_ropa)
            self._tiempo.cantidad = cantidad
        elif filetype == "Error":
            cantidad = self._tiempo.num_error
        else:
            log(
                INFO,
                f"El archivo de tipo {filetype} no está admitido. Solo se aceptan los valores Data y Error",
            )
            return

        datetime_obj = datetime.strptime(self._tiempo.fecha, "%d/%m/%Y")
        filepath = path.join(folder, datetime_obj.strftime("%d-%m-%Y"))
        filename = (
            filename
            + "_"
            + datetime_obj.strftime("%d%m%Y")
            + "_"
            + str(cantidad)
            + ".xlsx"
        )
        if not path.exists(filepath):
            makedirs(filepath)
        df_fb_mkp_ropa.to_excel(path.join(filepath, filename), index=False)
        log(INFO, f"{filetype} Guardados Correctamente")

    def guardar_tiempos(self, filename, sheet_name):
        """
        Guarda la información del tiempo de ejecución del scraper

        Parameters
        ----------
        filename: str
            Nombre del archivo
        sheet_name: str
            Nombre de la hoja de cálculo

        Returns
        -------
        None
        """
        log(INFO, "Guardando tiempos")
        self._tiempo.set_param_final()
        header_exist = True
        if path.isfile(filename):
            tiempos = load_workbook(filename)
            if sheet_name not in [ws.title for ws in tiempos.worksheets]:
                tiempos.create_sheet(sheet_name)
                header_exist = False
        else:
            tiempos = Workbook()
            tiempos.create_sheet(sheet_name)
            header_exist = False
        worksheet = tiempos[sheet_name]
        if not header_exist:
            worksheet.append(list(self._tiempo.__dict__.keys())[1:])
        worksheet.append(list(self._tiempo.__dict__.values())[1:])
        tiempos.save(filename)
        tiempos.close()
        log(INFO, "Tiempos Guardados Correctamente")

In [6]:
def config_log(log_folder, log_filename, log_file_mode, log_file_encoding, fecha_actual):
    """
    Función que configura los logs para rastrear al programa
        Parameter:
                log_folder (str): Carpeta donde se va a generar el archivo log
                log_filename (str): Nombre del archivo log a ser generado
                fecha_actual (datetime): Fecha actual de la creación del archivo log
        Returns:
                None
    """
    seleniumLogger.setLevel(ERROR)
    urllibLogger.setLevel(ERROR)
    logger = getLogger("seleniumwire")
    logger.setLevel(ERROR)
    log_path = path.join(log_folder, fecha_actual.strftime("%d-%m-%Y"))
    log_filename = log_filename + "_" + fecha_actual.strftime("%d%m%Y") + ".log"
    if not path.exists(log_path):
        makedirs(log_path)
    basicConfig(
        format="%(asctime)s %(message)s",
        level=INFO,
        handlers=[StreamHandler(), FileHandler(path.join(log_path, log_filename), log_file_mode, log_file_encoding)],
    )


def validar_parametros(parametros):
    """
    Función que valida si los parámetros a usar están definidos
         Parameter:
                 parametros (list): Lista de parámetros

        Returns:
               None
    """
    for parametro in parametros:
        if not parametro:
            log(ERROR, "Parámetros incorrectos")
            return False
    log(INFO, "Parámetros válidos")
    return True

In [7]:
def main():
    # Formato para el debugger
    fecha_actual = datetime.now().date() - timedelta(days=1)
    config_log("Log", "fb_ropa_log", "w", "utf-8", fecha_actual)
    log(INFO, "Configurando Formato Básico del Debugger")

    # Cargar variables de entorno
    log(INFO, "Cargando Variables de entorno")
    load_dotenv()

    # Url de la categoría a scrapear
    url_ropa = getenv("URL_CATEGORY")

    # Parámetros para guardar la data extraída por el scraper
    data_filename = getenv("DATA_FILENAME")
    data_folder = getenv("DATA_FOLDER")

    # Parámetros para guardar la medición de la ejecución del scraper
    filename_tiempos = getenv("FILENAME_TIEMPOS")
    sheet_tiempos = getenv("SHEET_TIEMPOS")

    # Parámetros para guardar los errores durante la ejecución por el scraper
    error_filename = getenv("ERROR_FILENAME")
    error_folder = getenv("ERROR_FOLDER")

    # Validar parámetros
    if not validar_parametros(
        [
            url_ropa,
            data_filename,
            data_folder,
            filename_tiempos,
            sheet_tiempos,
            error_filename,
            error_folder,
        ]
    ):
        return

    # Inicializar scrapper
    scraper = ScraperFb(fecha_actual)

    # Iniciar sesión
    scraper.iniciar_sesion()

    # Extracción de datos
    scraper.mapear_datos(url_ropa)

    # Guardando la data extraída por el scraper
    scraper.guardar_datos(scraper.data.dataset, "Data", data_folder, data_filename)

    # Guardando los errores extraídos por el scraper
    scraper.guardar_datos(
        scraper.errores.errores, "Error", error_folder, error_filename
    )

    # Guardando los tiempos durante la ejecución del scraper
    scraper.guardar_tiempos(filename_tiempos, sheet_tiempos)
    log(INFO, "Programa finalizado")

In [8]:
if __name__ == "__main__":
    main()

2023-01-13 23:13:30,452 Configurando Formato Básico del Debugger
2023-01-13 23:13:30,452 Cargando Variables de entorno
2023-01-13 23:13:30,452 Parámetros válidos
2023-01-13 23:13:30,452 Inicializando scraper
2023-01-13 23:13:30,452 Hora de inicio: 23:13:30
2023-01-13 23:13:30,468 ====== WebDriver manager ======
2023-01-13 23:13:31,389 Get LATEST chromedriver version for google-chrome 108.0.5359
2023-01-13 23:13:32,765 Driver [C:\Users\param\.wdm\drivers\chromedriver\win32\108.0.5359\chromedriver.exe] found in cache
2023-01-13 23:13:34,334 Iniciando sesión
2023-01-13 23:13:40,014 Inicio de sesión con éxito
2023-01-13 23:13:50,031 Accediendo a la URL
2023-01-13 23:14:16,600 Mapeando Publicaciones
2023-01-13 23:14:16,912 Scrapeando item 1
2023-01-13 23:14:22,125 En remate
2023-01-13 23:14:22,141 Item 1 scrapeado con éxito
2023-01-13 23:14:24,241 -------------------------------------------------------------------
2023-01-13 23:14:24,241 Scrapeando item 2
2023-01-13 23:14:29,455 Closet Sale

2023-01-13 23:16:11,215 Item 16 scrapeado con éxito
2023-01-13 23:16:13,279 -------------------------------------------------------------------
2023-01-13 23:16:13,295 Scrapeando item 17
2023-01-13 23:16:18,474 Zapatillas Reebok 24.5
2023-01-13 23:16:18,474 Item 17 scrapeado con éxito
2023-01-13 23:16:20,552 -------------------------------------------------------------------
2023-01-13 23:16:20,552 Scrapeando item 18
2023-01-13 23:16:25,762 Hermoso polo TOPY TOP
2023-01-13 23:16:25,762 Item 18 scrapeado con éxito
2023-01-13 23:16:27,926 -------------------------------------------------------------------
2023-01-13 23:16:27,926 Scrapeando item 19
2023-01-13 23:16:33,121 Zapatillas Talla 36
2023-01-13 23:16:33,121 Item 19 scrapeado con éxito
2023-01-13 23:16:35,170 -------------------------------------------------------------------
2023-01-13 23:16:35,170 Scrapeando item 20
2023-01-13 23:16:40,368 REMATO ZAPATILLAS HYPNOTIC
2023-01-13 23:16:40,368 Item 20 scrapeado con éxito
2023-01-13 2

2023-01-13 23:17:04,004 -------------------------------------------------------------------
2023-01-13 23:17:04,004 Scrapeando item 23
2023-01-13 23:17:09,159 Zapatillas Adidas Pride 👈💯 originales 👈💯 Stan Smith 👈
--- Logging error ---
Traceback (most recent call last):
  File "D:\Anaconda\envs\fb-ropa-env\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
  File "D:\Anaconda\envs\fb-ropa-env\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode characters in position 48-49: character maps to <undefined>
Call stack:
  File "D:\Anaconda\envs\fb-ropa-env\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\Anaconda\envs\fb-ropa-env\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda\envs\fb-ropa-env\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
 

2023-01-13 23:17:40,229 -------------------------------------------------------------------
2023-01-13 23:17:40,229 Scrapeando item 28
2023-01-13 23:17:45,386 Remate de vestidos dama
2023-01-13 23:17:45,386 Item 28 scrapeado con éxito
2023-01-13 23:17:47,439 -------------------------------------------------------------------
2023-01-13 23:17:47,439 Scrapeando item 29
2023-01-13 23:17:52,612 Lindas medias damas 
Kitty , melody , keropi y chococat
2023-01-13 23:17:52,612 Item 29 scrapeado con éxito
2023-01-13 23:17:54,676 -------------------------------------------------------------------
2023-01-13 23:17:54,676 Scrapeando item 30
2023-01-13 23:17:59,824 Vestido tendencias verano
2023-01-13 23:17:59,824 Item 30 scrapeado con éxito
2023-01-13 23:18:01,887 -------------------------------------------------------------------
2023-01-13 23:18:01,887 Scrapeando item 31
2023-01-13 23:18:07,055 Vestido
2023-01-13 23:18:07,055 Item 31 scrapeado con éxito
2023-01-13 23:18:09,163 ------------------

2023-01-13 23:20:02,633 Vestidos largos full licra
2023-01-13 23:20:02,633 Item 46 scrapeado con éxito
2023-01-13 23:20:04,674 -------------------------------------------------------------------
2023-01-13 23:20:04,674 Scrapeando item 47
2023-01-13 23:20:09,809 Botines sybilla talla 39 con un par de usos
2023-01-13 23:20:09,809 Item 47 scrapeado con éxito
2023-01-13 23:20:11,867 -------------------------------------------------------------------
2023-01-13 23:20:11,868 Scrapeando item 48
2023-01-13 23:20:17,027 Closet sale 🌞 Prendas de Remate 🌻
--- Logging error ---
Traceback (most recent call last):
  File "D:\Anaconda\envs\fb-ropa-env\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
  File "D:\Anaconda\envs\fb-ropa-env\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f31e' in position 36: character maps to <undefined>


...

2023-01-13 23:42:36,123 Scrapeando item 225
2023-01-13 23:42:41,292 hermosos polivestidos sublimados
2023-01-13 23:42:41,292 Item 225 scrapeado con éxito
2023-01-13 23:42:43,366 -------------------------------------------------------------------
2023-01-13 23:42:43,368 Scrapeando item 226
2023-01-13 23:42:48,524 Overol Jean talla S-M
2023-01-13 23:42:48,524 Item 226 scrapeado con éxito
2023-01-13 23:42:50,592 -------------------------------------------------------------------
2023-01-13 23:42:50,592 Scrapeando item 227
2023-01-13 23:42:55,740 Zapatillas fila para dama
2023-01-13 23:42:55,740 Item 227 scrapeado con éxito
2023-01-13 23:42:57,809 -------------------------------------------------------------------
2023-01-13 23:42:57,811 Scrapeando item 228
2023-01-13 23:43:02,959 Vestidos elegantes tallas S,M y L 
Precio 50 soles nuevo
2023-01-13 23:43:02,959 Item 228 scrapeado con éxito
2023-01-13 23:43:05,043 -------------------------------------------------------------------
2023-01-13

2023-01-13 23:43:33,979 -------------------------------------------------------------------
2023-01-13 23:43:33,980 Scrapeando item 233
2023-01-13 23:43:39,127 Blazer Low plataforma 🔥 talla 39
--- Logging error ---
Traceback (most recent call last):
  File "D:\Anaconda\envs\fb-ropa-env\lib\logging\__init__.py", line 1084, in emit
    stream.write(msg + self.terminator)
  File "D:\Anaconda\envs\fb-ropa-env\lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f525' in position 46: character maps to <undefined>
Call stack:
  File "D:\Anaconda\envs\fb-ropa-env\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "D:\Anaconda\envs\fb-ropa-env\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "D:\Anaconda\envs\fb-ropa-env\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.laun

2023-01-13 23:43:55,679 -------------------------------------------------------------------
2023-01-13 23:43:55,681 Scrapeando item 236
2023-01-13 23:44:00,858 Vestido Cafarena Rib
2023-01-13 23:44:00,858 Item 236 scrapeado con éxito
2023-01-13 23:44:02,932 -------------------------------------------------------------------
2023-01-13 23:44:02,935 Scrapeando item 237
2023-01-13 23:44:08,116 CASACA UMBRO ORIGINAL
2023-01-13 23:44:08,116 Item 237 scrapeado con éxito
2023-01-13 23:44:10,225 -------------------------------------------------------------------
2023-01-13 23:44:10,226 Scrapeando item 238
2023-01-13 23:44:15,401 Vestido H&M Manga Larga
2023-01-13 23:44:15,417 Item 238 scrapeado con éxito
2023-01-13 23:44:17,496 -------------------------------------------------------------------
2023-01-13 23:44:17,497 Scrapeando item 239
2023-01-13 23:44:22,662 Enterizo de Dama
2023-01-13 23:44:22,662 Item 239 scrapeado con éxito
2023-01-13 23:44:24,738 ----------------------------------------

2023-01-13 23:46:13,565 Scrapeando item 254
2023-01-13 23:46:18,736 Vestidos
2023-01-13 23:46:18,737 Item 254 scrapeado con éxito
2023-01-13 23:46:20,820 -------------------------------------------------------------------
2023-01-13 23:46:20,821 Scrapeando item 255
2023-01-13 23:46:26,002 PALAZOS
2023-01-13 23:46:26,018 Item 255 scrapeado con éxito
2023-01-13 23:46:28,125 -------------------------------------------------------------------
2023-01-13 23:46:28,126 Scrapeando item 256
2023-01-13 23:46:33,327 Zapatilla nike dunk low talla 42
2023-01-13 23:46:33,327 Item 256 scrapeado con éxito
2023-01-13 23:46:35,432 -------------------------------------------------------------------
2023-01-13 23:46:35,433 Scrapeando item 257
2023-01-13 23:46:40,593 Vestidos cóctel desde 50
2023-01-13 23:46:40,593 Item 257 scrapeado con éxito
2023-01-13 23:46:42,665 -------------------------------------------------------------------
2023-01-13 23:46:42,666 Scrapeando item 258
2023-01-13 23:46:47,844 Linda

2023-01-13 23:47:55,664 -------------------------------------------------------------------
2023-01-13 23:47:55,665 Scrapeando item 268
2023-01-13 23:48:00,843 Zapatilla Nike
2023-01-13 23:48:00,858 Item 268 scrapeado con éxito
2023-01-13 23:48:02,940 -------------------------------------------------------------------
2023-01-13 23:48:02,941 Scrapeando item 269
2023-01-13 23:48:08,143 CONJUNTO
2023-01-13 23:48:08,159 Item 269 scrapeado con éxito
2023-01-13 23:48:10,237 -------------------------------------------------------------------
2023-01-13 23:48:10,238 Scrapeando item 270
2023-01-13 23:48:15,420 Vestidos verano closet sale
2023-01-13 23:48:15,420 Item 270 scrapeado con éxito
2023-01-13 23:48:24,549 -------------------------------------------------------------------
2023-01-13 23:48:24,549 Se halló 0 errores
2023-01-13 23:48:24,565 Fin de la extraccion
2023-01-13 23:48:24,565 Guardando Data
2023-01-13 23:48:24,783 Data Guardados Correctamente
2023-01-13 23:48:24,783 Guardando Err